In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import glob
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
import segmentation_models as sm
sm.set_framework('tf.keras')
import multiprocessing as mp
from save_weights_every_epoch import CallbackForSavingModelWeights
from change_learning_rate_epoch import ChangeLR
import multiprocessing as mp

Segmentation Models: using `keras` framework.


In [3]:
load_dotenv('../config_files/dev.env')

True

In [4]:
csv_file = pd.read_csv('../../files/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [5]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [6]:
for x in tqdm(csv_file['fixed_inputs'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 387183.10it/s]


In [7]:
for x in tqdm(csv_file['mask_paths'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 402124.45it/s]


In [8]:
csv_file['fixed_inputs'].values.tolist()[0]

'../../files/train_v2/00003e153.jpg'

In [9]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../files/train_v2/6384c3e78.jpg']

In [10]:
allowed_gpus = [0, 1, 2, 3, 4, 5, 6, 7]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [11]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size, random_state=42)
    train, val = train_test_split(train, test_size = test_size, random_state=42)
    return train, val, test

In [12]:
# csv_file = pd.read_csv
train, val, test = split_datasets(csv_file)

In [13]:
def get_model(model_name=None, image_size=524):
    with strategy.scope():
        if model_name == None:
            model = sm.PSPNet(classes=1, input_shape=(image_size, image_size, 3), activation='sigmoid')
        else:
            model = sm.PSPNet(model_name, weights=None, input_shape=(image_size, image_size, 3), classes=1, activation='sigmoid')
        model.compile(
            tf.keras.optimizers.SGD(),
            loss = sm.losses.bce_dice_loss,
            metrics = [sm.metrics.iou_score]
        )
    return model

In [14]:
def read_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels=1)
    img = tf.image.resize(img, size=shape)
    mask = tf.image.resize(mask, size=shape)
    img = img / 255
    mask = mask / 255
    return img, mask

In [15]:
def get_data(data, test_ = False, aug_seq = None, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [16]:
os.environ['tb_path']

'/home/ubuntu/ship_segmentation/TB/'

In [16]:
K.clear_session()
batch_size = 64
image_size = 528
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/res50_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/res50_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model(model_name='resnet50', image_size=image_size)
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [17]:
K.clear_session()
batch_size = 64
image_size = 528
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_incep_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_incep_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model(model_name='inceptionv3', image_size=image_size)
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback, weights_save]
    )

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 69 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 69 all-reduces with algorithm = nccl, num_packs = 1
368/368 [==============================] - 180s 379ms/step - loss: 0.9999 - iou_score: 0.0064 - val_loss: 0.8821 - val_iou_score: 0.0657
Epoch 2/50
368/368 [==============================] - 137s 372ms/step - loss: 0.4153 - iou_score: 0.4337 - val_loss: 0.3564 - val_iou_score: 0.4928
Epoch 3/50
368/368 [==============================] - 136s 371ms/step - loss: 0.2945 - iou_score: 0.5576 - val_loss: 0.3085 - val_iou_score: 0.5449
Epoch 4/50
368/368 [==============================] - 136s 371ms/step - loss: 0.2646 - iou_score: 0.5935 - val_loss: 0.2888 - val_iou_score: 0.5710
Epoch 5/50
368/368 [==============================] - 137s 371ms/step - loss: 0.2448 - iou_score: 0.6180 - val_loss: 0.2644 - val_iou_score: 0.5991
Epoch 6/50
368/368 [==============================] - 137s 371ms/step - loss: 0.2315 - i

In [17]:
K.clear_session()
batch_size = 64
image_size = 528
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model(image_size=image_size)
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback, weights_save]
    )

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu